In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [10]:
# Code Generation
message = "1101"  # 4-bit binary message

nr_codewords = int(100)
bits_info = torch.randint(2, (nr_codewords, 4), dtype=torch.float)

print(bits_info)

tensor([[1., 0., 1., 0.],
        [1., 1., 1., 1.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 1.],
        [1., 0., 1., 1.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 0.],
        [1., 1., 0., 1.],
        [1., 1., 1., 0.],
        [0., 1., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 1., 0.],
        [1., 1., 1., 1.],
        [0., 1., 1., 1.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 1., 1.],
        [1., 1., 0., 1.],
        [1., 1., 0., 1.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [1., 0., 0., 1.],
        [0., 0., 1., 0.],
        [1., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 0., 1.],
        [0., 0., 1., 1.],
        [0.,

Hamming(7,4) Encoder

In [11]:
class hamming_encode(torch.nn.Module):
    def __init__(self):
        """
            Use Hamming(7,4) to encode the data.
    
        Args:
            data: data received from the Hamming(7,4) encoder(Tensor)
            generator matrix: generate the parity code
    
        Returns:
            encoded data: 4 bits original info with 3 parity code.
        """
        super(hamming_encode, self).__init__()

        # Define the generator matrix for Hamming(7,4)
        self.generator_matrix = torch.tensor([
            [1, 0, 0, 0],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1],
            [1, 1, 0, 1],
            [1, 0, 1, 1],
            [0, 1, 1, 1],
        ], dtype=torch.float)

    def forward(self, input_data):
        # Ensure input_data has shape (batch_size, 4)
        assert input_data.size(1) == 4, "Input data must have 4 bits."

        # Perform matrix multiplication to encode the data
        encoded_data = torch.matmul(input_data, self.generator_matrix.t()) % 2

        return encoded_data

In [20]:
encoder = hamming_encode()
encoded_codeword = encoder(bits_info)
print(encoded_codeword)

tensor([[1., 0., 1., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 0., 1., 1., 0., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1.],
        [1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 1., 1.],
        [1., 1., 0., 1., 1., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1.],
        [0., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0.],
        [0., 0., 0., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 0., 0.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 1., 0., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.],
        [0., 1., 1., 1., 0., 0., 1.],
        [0., 1., 0., 0., 1., 0., 1.],
        [1.,

BPSK Modulator + Noise

In [48]:
class bpsk_modulator(torch.nn.Module):
    def __init__(self):
        """
        Use BPSK to compress the data, which is easily to transmit.

    Args:
        data: data received from the Hamming(7,4) encoder(Tensor)
        symbol_rate: Symbol rate in Hz
        carrier_freq: Carrier frequency in Hz
        snr_dB: Signal-to-noise ratio in dB

    Returns:
        time:
        data: Tensor contain all data modulated and add noise
    """
        super(bpsk_modulator, self).__init__()
        
    def forward(self, data, symbol_rate, carrier_freq, snr_dB):
    
        for i in range(data.shape[0]):
            bits = data[i]
            bits = 2 * bits - 1
            # Time vector
            time = torch.arange(0, len(bits) / symbol_rate, 1 / symbol_rate)
        
            # Generate carrier signal
            carrier = torch.cos(2 * torch.pi * carrier_freq * time)
        
            # Modulate the signal
            modulated_signal = bits * carrier
        
            # Add Gaussian noise to the signal
            noise_power = torch.tensor(10**(-snr_dB / 10))
            noise = torch.sqrt(noise_power) * torch.randn(len(modulated_signal))
            noised_signal = modulated_signal + noise
            data[i] = noised_signal
    
       
        return time, data

In [49]:
data = encoded_codeword  # Binary data
symbol_rate = 15  # Symbol rate in Hz
carrier_freq = 100  # Carrier frequency in Hz
snr_dB = 15  # Signal-to-noise ratio in dB

# Modulate the signal
modulator = bpsk_modulator()
time, modulated_noise_signal = modulator(data, symbol_rate, carrier_freq, snr_dB)
print(modulated_noise_signal)

tensor([[-2.7870e+02,  8.8519e-01,  9.2498e-01, -3.9484e+03,  1.1469e+00,
         -7.2990e-01,  6.1791e+02],
        [ 1.2190e+01,  3.4575e-01,  6.4224e-01,  3.0237e+01,  1.1055e+00,
          1.1407e+00, -2.7931e+01],
        [-2.4650e+02, -7.0381e-01,  1.1112e+00, -2.6680e+02, -2.0623e-01,
          1.0032e+00, -1.1356e+01],
        [-2.5567e+02,  6.3732e-01,  2.6231e-01,  3.9871e+01,  6.0089e-01,
          1.4025e+00,  2.7707e+01],
        [-2.9820e+02, -8.2562e-01,  1.6308e+00, -2.6454e+02,  4.8845e-01,
          1.0059e+00, -1.6088e+01],
        [-3.4758e+01,  7.7128e-01, -7.2464e-01, -1.1906e+01,  1.2282e+00,
         -1.3251e-01, -2.8403e+02],
        [ 6.4629e+00,  4.7527e-02,  9.5357e-01,  2.3994e+01, -1.4758e-01,
          1.6873e+00, -3.0481e+02],
        [ 2.3464e+01,  8.3905e-01,  1.1195e+00,  2.3438e+01,  1.2328e+00,
          7.6068e-01,  9.0780e+00],
        [-2.8852e+02,  2.2956e-01,  1.2798e+00,  1.2024e+01,  6.1349e-01,
          1.6192e+00, -2.9966e+01],
        [ 

LLR Log-likelihood

In [51]:
def llr(signal, noise_std):
    """
    Calculate Log Likelihood Ratio (LLR) for a simple binary symmetric channel.

    Args:
        signal (torch.Tensor): Received signal.
        noise_std (float): Standard deviation of the noise.

    Returns:
        torch.Tensor: Log Likelihood Ratio (LLR) values.
    """
    # Assuming Binary Phase Shift Keying (BPSK) modulation
    likelihood_0 = -0.5 * ((signal - 1) / noise_std)**2
    likelihood_1 = -0.5 * ((signal + 1) / noise_std)**2

    # Calculate the LLR
    llr_values = likelihood_1 - likelihood_0

    return llr_values

# Example usage:
received_signal = torch.tensor([-1.5, 0.8, 1.2, -0.5])
noise_std_dev = 0.6

llr_result = llr(received_signal, noise_std_dev)
print("LLR values:", llr_result)

LLR values: tensor([ 8.3333, -4.4444, -6.6667,  2.7778])


In [147]:
# import numpy as np
# 
# def ldpc_decode(received_codeword, max_iterations=50):
#     # LDPC Parity-check matrix (7,4) example
#     H = np.array([
#         [1, 0, 1, 1, 0, 0, 0],
#         [0, 1, 0, 0, 1, 0, 1],
#         [0, 0, 0, 1, 0, 1, 0]
#     ])
# 
#     # LDPC Generator matrix (7,4) example
#     G = np.array([
#         [1, 0, 0, 0],
#         [0, 1, 0, 0],
#         [0, 0, 1, 0],
#         [0, 0, 0, 1],
#         [0, 1, 1, 1],
#         [1, 0, 1, 1],
#         [1, 1, 0, 1]
#     ])
# 
#     # Initialize the received codeword and the decoded message
#     y = np.array(list(map(int, received_codeword)))
#     x_hat = np.zeros(G.shape[1])
# 
#     for iteration in range(max_iterations):
#         # Syndrome check
#         syndrome = np.mod(np.dot(H, y), 2)
# 
#         if np.all(syndrome == 0):
#             # If the syndrome is all zeros, the decoding is successful
#             break
# 
#         # Compute the log-likelihood ratios (LLRs) using the sum-product algorithm
#         llrs = np.zeros(G.shape[1])
#         for j in range(G.shape[1]):
#             connected_checks = np.nonzero(H[:, j])[0]
#             llrs[j] = 2 * y[j] / np.tanh(np.sum(np.arctanh(np.tanh(y[i] / 2)) for i in connected_checks))
# 
#         # Make hard decisions based on LLRs
#         x_hat = np.where(llrs > 0, 1, 0)
# 
#         # Update the received codeword
#         y = np.mod(np.dot(G, x_hat) + y, 2)
# 
#     decoded_message = ''.join(map(str, x_hat.astype(int)))
#     return decoded_message
# 
# def main():
#     # Example usage
#     received_codeword = "1101001"  # Received codeword with possible errors
#     decoded_message = ldpc_decode(received_codeword)
# 
#     print(f"Received codeword: {received_codeword}")
#     print(f"Decoded message: {decoded_message}")
# 
# if __name__ == "__main__":
#     main()
